# Test for one label :
- create matchup file
- add low pass filter
- add sum except 
- compute MS

In [6]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import os
from glob import glob

import histlib.erastar as match

from histlib.cstes import labels, zarr_dir
zarr_dir_ok = zarr_dir+'_ok'

In [2]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    from dask import config
    config.set({"distributed.comm.timeouts.connect": "200s"})
    cluster = PBSCluster(cores=5, processes=5, walltime='04:00:00')
    #cluster = PBSCluster(cores=20, processes=20, walltime='02:00:00')#8
    w = cluster.scale(jobs=5)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.238:8787/status,
Dashboard: http://10.148.0.238:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.238:52665,Workers: 0
Dashboard: http://10.148.0.238:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


_______________
# One label
## Matchup

In [3]:
from histlib.matchup import matchup_dataset_one

In [4]:
l = labels[0]
dsm = matchup_dataset_one(l).persist()

## Add filter 

In [6]:
from histlib.matchup import add_low_pass_filter_to_matchup
dsfm = add_low_pass_filter_to_matchup(l, T=2, cutoff=4) 

In [7]:
dsfm

<xarray.Dataset>
Dimensions:               (obs: 36694)
Coordinates:
  * obs                   (obs) int64 0 2 3 4 5 ... 43068 43069 43071 43072
    lat                   (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    lon                   (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    time                  (obs) datetime64[ns] dask.array<chunksize=(500,), meta=np.ndarray>
Data variables:
    drifter_acc_x_4       (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_acc_y_4       (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_coriolis_x_4  (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
    drifter_coriolis_y_4  (obs) float64 dask.array<chunksize=(500,), meta=np.ndarray>
Attributes:
    __id:                   CMEMS-L3-SLA-Jason-3__gps
    __time_coverage_end:    2020-05-01T00:00:00
    __time_coverage_start:  2020-01-01T00:00:00

In [8]:
ds = xr.merge([dsm, dsfm])

## Add sum except
### Default

In [9]:
from histlib.matchup import add_except_sum
dss = add_except_sum(ds)

dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'es_cstrio_z0_alti_wd_x', 'co__0__sla_filtered__es_cstrio_z0__alti_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'es_cstrio_z0_drifter_wd_x', 'co__0__sla_filtered__es_cstrio_z0__drifter_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'e5_cstrio_z0_alti_wd_x', 'co__0__sla_filtered__e5_cstrio_z0__alti_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'e5_cstrio_z0_drifter_wd_x', 'co__0__sla_filtered__e5_cstrio_z0__drifter_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'es_cstrio_z15_alti_wd_x', 'co__0__sla_filtered__es_cstrio_z15__alti_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alti_ggx_sla_filtered', 'es_cstrio_z15_drifter_wd_x', 'co__0__sla_filtered__es_cstrio_z15__drifter_x'])
dict_values(['drifter_acc_x_0', 'drifter_coriolis_x_0', 'alt

### Select

In [17]:
from histlib.matchup import find_term_list
wd_x, wd_y, grad_x, grad_y, cutoff = find_term_list(dss)
print(wd_x)
print(wd_y)
print(grad_x)
print(grad_y)
print(cutoff)

['es_cstrio_z0_alti_wd_x', 'es_cstrio_z0_drifter_wd_x', 'e5_cstrio_z0_alti_wd_x', 'e5_cstrio_z0_drifter_wd_x', 'es_cstrio_z15_alti_wd_x', 'es_cstrio_z15_drifter_wd_x', 'e5_cstrio_z15_alti_wd_x', 'e5_cstrio_z15_drifter_wd_x']
['es_cstrio_z0_alti_wd_y', 'es_cstrio_z0_drifter_wd_y', 'e5_cstrio_z0_alti_wd_y', 'e5_cstrio_z0_drifter_wd_y', 'es_cstrio_z15_alti_wd_y', 'es_cstrio_z15_drifter_wd_y', 'e5_cstrio_z15_alti_wd_y', 'e5_cstrio_z15_drifter_wd_y']
['alti_ggx_sla_filtered', 'alti_ggx_sla_unfiltered', 'alti_ggx_sla_unfiltered_denoised', 'alti_ggx_adt_filtered', 'alti_ggx_adt_unfiltered', 'alti_ggx_adt_unfiltered_denoised', 'aviso_alti_ggx_adt', 'aviso_drifter_ggx_adt', 'aviso_alti_ggx_sla', 'aviso_drifter_ggx_sla']
['aviso_alti_ggy_adt', 'aviso_drifter_ggy_adt', 'aviso_alti_ggy_sla', 'aviso_drifter_ggy_sla']
['0', '4']


In [18]:
dsss = add_except_sum(ds, wd_x =wd_x[:2], wd_y =wd_y[:2], grad_x = grad_x[:2], grad_y = grad_y[:2], cutoff=['4'])

dict_values(['drifter_acc_x_4', 'drifter_coriolis_x_4', 'alti_ggx_sla_filtered', 'es_cstrio_z0_alti_wd_x', 'co__4__sla_filtered__es_cstrio_z0__alti_x'])
dict_values(['drifter_acc_x_4', 'drifter_coriolis_x_4', 'alti_ggx_sla_filtered', 'es_cstrio_z0_drifter_wd_x', 'co__4__sla_filtered__es_cstrio_z0__drifter_x'])
dict_values(['drifter_acc_x_4', 'drifter_coriolis_x_4', 'alti_ggx_sla_unfiltered', 'es_cstrio_z0_alti_wd_x', 'co__4__sla_unfiltered__es_cstrio_z0__alti_x'])
dict_values(['drifter_acc_x_4', 'drifter_coriolis_x_4', 'alti_ggx_sla_unfiltered', 'es_cstrio_z0_drifter_wd_x', 'co__4__sla_unfiltered__es_cstrio_z0__drifter_x'])
dict_values(['drifter_acc_y_4', 'drifter_coriolis_y_4', 'aviso_alti_ggy_adt', 'es_cstrio_z0_alti_wd_y', 'aviso__4__adt__es_cstrio_z0__alti_y'])
dict_values(['drifter_acc_y_4', 'drifter_coriolis_y_4', 'aviso_drifter_ggy_adt', 'es_cstrio_z0_drifter_wd_y', 'aviso__4__adt__es_cstrio_z0__drifter_y'])


## Compute MS

In [21]:
dsms = (dsss**2).mean('obs')

In [8]:
cluster.close()